In [95]:
import os
import data
import tensorflow.keras as keras
import os, sys

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt


def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

root_logdir = os.path.join(os.curdir, "my_logs")
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)




In [96]:
o = data.Train_Data('D:/DS/Learnin/Essex MS/CE888/git/CE888/assignment2/seq2seq/spider/',['train_spider.json','train_others.json'])

o.sql[0]

Reading file at path D:/DS/Learnin/Essex MS/CE888/git/CE888/assignment2/seq2seq/spider/train_spider.json
7000 Rows in Total
Reading file at path D:/DS/Learnin/Essex MS/CE888/git/CE888/assignment2/seq2seq/spider/train_others.json
8659 Rows in Total
Filter Easy Queries
Splittin the Train and Test data
(727, 7)
Data for Training (2908, 7)
Data for Testing (727, 7)


'SELECT Date_of_ceremony ,  RESULT FROM music_festival'

In [112]:
input_sentences = [data.Preprocess(x).processed_text.numpy().decode().replace(data.SOS,'').replace(data.EOS,'') for x in o.questions]
output_sentences = [data.Preprocess(x).processed_text.numpy().decode().replace(data.SOS,'') for x in o.sql]
output_sentences_inputs = [data.Preprocess(x).processed_text.numpy().decode().replace(data.EOS,'') for x in o.questions]

In [47]:
BATCH_SIZE = 64
EPOCHS = 20
LSTM_NODES =256
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 256

In [49]:
print(input_sentences[172])
print(output_sentences[172])
print(output_sentences_inputs[172])

what is the highest elevation in the united states
select max  (  highest_elevation  )  from highlow; [end]
[start] select max  (  highest_elevation  )  from highlow;


In [115]:
print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))
print("num samples output input:", len(output_sentences_inputs))

num samples input: 2908
num samples output: 2908
num samples output input: 2908


In [116]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 2163
Length of longest sentence in input: 38


In [118]:
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 3601
Length of longest sentence in the output: 58


In [125]:
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
print("encoder_input_sequences[172]:", encoder_input_sequences[172])

encoder_input_sequences.shape: (2908, 38)
encoder_input_sequences[172]: [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   3   8   1  72 316   6   1
 237  51]


In [126]:
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])

decoder_input_sequences.shape: (2908, 58)
decoder_input_sequences[172]: [  5  14  23   2 122 311  16   2 466  95   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]


In [142]:
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_output_sequences.shape:", decoder_output_sequences.shape)
print("decoder_output_sequences[172]:", decoder_output_sequences[172])

decoder_output_sequences.shape: (2908, 58)
decoder_output_sequences[172]: [   3   55    8  478    9    1 1061    4    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [143]:
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, input_length=max_input_len)

In [144]:
decoder_targets_one_hot = np.zeros((
        len(input_sentences),
        max_out_len,
        num_words_output
    ),
    dtype='float32'
)

In [145]:
decoder_targets_one_hot.shape


(2908, 58, 3602)

In [156]:
# decoder_targets_one_hot[172,1,55]

In [157]:
for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        # print(i,t,word)
        decoder_targets_one_hot[i, t, word] = 1

In [158]:
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [159]:
decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

In [160]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [166]:
model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [211]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=64,
    epochs=300,
    validation_split=0.1,callbacks = [tensorboard_cb])

Epoch 1/300
41/41 [==============================] - 39s 919ms/step - loss: 7.4265e-04 - accuracy: 0.9998 - val_loss: 0.9634 - val_accuracy: 0.9010
Epoch 2/300
41/41 [==============================] - 33s 817ms/step - loss: 0.0013 - accuracy: 0.9997 - val_loss: 0.9775 - val_accuracy: 0.9054
Epoch 3/300
41/41 [==============================] - 34s 823ms/step - loss: 6.7916e-04 - accuracy: 0.9998 - val_loss: 1.0294 - val_accuracy: 0.9013
Epoch 4/300
41/41 [==============================] - 35s 855ms/step - loss: 7.2842e-04 - accuracy: 0.9998 - val_loss: 0.9854 - val_accuracy: 0.9054
Epoch 5/300
41/41 [==============================] - 34s 826ms/step - loss: 5.7756e-04 - accuracy: 0.9998 - val_loss: 0.9922 - val_accuracy: 0.9049
Epoch 6/300
41/41 [==============================] - 34s 831ms/step - loss: 8.1995e-04 - accuracy: 0.9998 - val_loss: 1.0016 - val_accuracy: 0.9053
Epoch 7/300
41/41 [==============================] - 32s 786ms/step - loss: 9.9760e-04 - accuracy: 0.9997 - val_loss

In [212]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [213]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [214]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [215]:
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [216]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [217]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs[data.SOS]
    eos = word2idx_outputs[data.EOS]
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [218]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)

-
Input:  give the maximum price and score for wines produced in the appelation st .  helena . 
Response: select max ( price ) score ) from wine where price = wine


In [219]:
i

2035

In [220]:
encoder_input_sequences[i]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,   61,    1,   44,   64,    5,  137,   18,  134,  158,
          6,    1, 1251, 1249, 1250])

In [221]:


input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)


encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)


In [222]:
test_q = o.df_test['question'].values.tolist()
test_sql = o.df_test['query'].values.tolist()
test_input_sentences = [data.Preprocess(x).processed_text.numpy().decode().replace(data.SOS,'').replace(data.EOS,'') for x in test_q]

In [223]:
test_input_integer_seq = input_tokenizer.texts_to_sequences(test_input_sentences)

In [224]:
test_encoder_input_sequences = pad_sequences(test_input_integer_seq, maxlen=max_input_len)

In [225]:
test_encoder_input_sequences[100]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,   61,   55,    1,   10,    5,  109,    2,
          1,   84,   47,   56, 1764])

In [229]:
count = 0
for i in range(len(test_input_sentences)):
    test_input_seq = test_encoder_input_sequences[i:i+1]
    translation = translate_sentence(test_input_seq)
    print('-')
    print('Input:', test_input_sentences[i])
    print(test_sql[i])
    print('Response:', translation)
    pred = translation.lower().replace(' ','')
    act = test_sql[i].lower().replace(' ','')
    if pred == act:
            count = count +1
print("Accuracy is {:.2f} %".format(count*100/len(test_input_sentences)))

-
Input:  how many team franchises are active ,  with active value 'y' ? 
SELECT count(*) FROM team_franchise WHERE active = 'Y';
Response: select name from customer where how_to_get_there = 1990 and for = 1980;
-
Input:  how many products are never booked with amount higher than 200 ? 
SELECT count(*) FROM Products_for_hire WHERE product_id NOT IN ( SELECT product_id FROM products_booked WHERE booked_amount  >  200 )
Response: select department_id from employees where salary not ( ( select from from where where = = 02 or : 06 : : 06 : : : : 33 : : : : : : : : : : : : : : : : : : : : : : : : : : : : : :
-
Input:  what are the first and last names of all customers with between 1000 and 3000 dollars outstanding ? 
SELECT first_name ,  last_name FROM Customers WHERE amount_outstanding BETWEEN 1000 AND 3000;
Response: select date_of_transaction from faculty_participates_in intersect select from customers intersect select , , from where where = "football" and
-
Input:  what is the nickname 